На сайте работает динамическое создание ссылок при прокрутке первых 500 песен, так что я сначала выделил div блок songs, и затем уже сформировал json файл, в котором будет содержаться минимальная meta информация по песням и ссылки на табы в них

In [40]:
from bs4 import BeautifulSoup
import json

# Открываем скачанный HTML файл
with open('formatted.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Парсим содержимое HTML файла
soup = BeautifulSoup(html_content, 'html.parser')

# Ищем все блоки с песнями
song_blocks = soup.find_all('a', class_='B0cew B0c14s')

songs_data = []

# Проходим по каждому блоку с песней и извлекаем данные
for song in song_blocks:
    # Извлекаем название трека
    song_name = song.find('div', {'data-field': 'name'}).text.strip()
    # Извлекаем исполнителя
    artist_name = song.find('div', {'data-field': 'artist'}).text.strip()
    # Извлекаем ссылку на табы
    tab_link = 'https://www.songsterr.com' + song['href']
    
    # Формируем структуру данных для JSON
    songs_data.append({
        'song_name': song_name,
        'artist_name': artist_name,
        'tab_link': tab_link
    })

# Сохраняем данные в JSON файл
with open('songs_data.json', 'w') as json_file:
    json.dump(songs_data, json_file, indent=4)

print('Данные сохранены в songs_data.json')


Данные сохранены в songs_data.json


Дальше проходимся по json файлу и качаем guitar pro файлы

In [41]:
import os
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from urllib.parse import urlparse
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Путь к папке для сохранения файлов
download_folder = os.path.join(os.getcwd(), "downloads")
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Опции для Chrome
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Открытие браузера на полный экран

# Настройка для автоматического сохранения файлов в нужную папку без запроса
prefs = {
    "download.default_directory": download_folder,  # Указываем путь для загрузки
    "download.prompt_for_download": False,  # Отключаем диалог загрузки
    "download.directory_upgrade": True,  # Обновление пути загрузки при необходимости
    "safebrowsing.enabled": True  # Отключение безопасного поиска для автоматической загрузки
}
chrome_options.add_experimental_option("prefs", prefs)

# Настройка драйвера Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Загрузка JSON-файла с данными о песнях
json_file_path = "songs_data.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    songs = json.load(file)

# Переходим на страницу авторизации
driver.get("https://www.songsterr.com/a/wa/signin")

# Ждем некоторое время для полной загрузки страницы
time.sleep(5)

# Авторизация
try:
    email_input = driver.find_element(By.NAME, 'email')
    email_input.send_keys("udalov0078@mail.ru")
    password_input = driver.find_element(By.NAME, 'password')
    password_input.send_keys("Ar01937800")
    signin_button = driver.find_element(By.ID, 'signin')
    signin_button.click()
    print("Успешный вход.")
    time.sleep(5)  # Ждем завершения входа
except Exception as e:
    print("Ошибка при попытке входа:", str(e))

# Проходим по каждой песне из JSON и скачиваем табулатуры, а также ищем ссылку на YouTube
for song in songs:
    try:
        # Проверяем, есть ли уже скачанный файл и ссылка на YouTube
        if "downloaded_file" in song and song["downloaded_file"] and "youtube_link" in song and song["youtube_link"]:
            print(f"Песня {song['song_name']} уже обработана, пропускаем.")
            continue  # Пропускаем уже обработанные песни

        # Переходим на страницу с табулатурой
        driver.get(song["tab_link"])
        time.sleep(5)  # Ожидаем загрузку страницы

        # Нажимаем кнопку для открытия списка ревизий
        button = driver.find_element(By.ID, 'revisions-toggle-tab')
        button.click()
        print(f"Кнопка ревизий для {song['song_name']} нажата.")
        time.sleep(3)  # Ожидаем загрузку доп. меню

        # Ищем первый элемент li внутри #revisions-list и ссылку для скачивания
        revisions_list = driver.find_element(By.ID, 'revisions-list')
        first_revision_item = revisions_list.find_element(By.TAG_NAME, 'li')
        download_link = first_revision_item.find_element(By.ID, 'download-link')
        download_url = download_link.get_attribute('href')

        # Скачиваем файл в папку "downloads"
        driver.get(download_url)
        print(f"Скачивание файла для {song['song_name']} начато: {download_url}")

        # Добавляем путь до файла в папку downloads
        file_path = os.path.join("downloads/", download_url)

        # Добавляем путь до файла в JSON
        song["downloaded_file"] = file_path

        # Ищем ссылку на YouTube видео
        try:
            # Нажимаем на кнопку для открытия iframe с YouTube-видео
            play_button = driver.find_element(By.ID, 'control-play')
            play_button.click()
            time.sleep(3)  # Ожидаем некоторое время для начала воспроизведения

            try:
                # Ждем появления контейнера с YouTube видео
                youtube_container = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'youtube-video-container'))
                )

                # Ожидаем, пока iframe станет доступен и переключаемся на него
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, 'iframe'))
                )

                # Получаем ссылку на видео из iframe
                youtube_url = iframe.get_attribute('src')  # После переключения можно сразу получить текущий URL
                print(f"YouTube видео найдено: {youtube_url}")
                
                # Возвращаемся на исходную страницу
                driver.switch_to.default_content()

                # Добавляем ссылку на YouTube в JSON
                song["youtube_link"] = youtube_url
                print(f"Ссылка на YouTube для {song['song_name']} найдена: {youtube_url}")

            except Exception as e:
                print(f"Не удалось найти YouTube видео: {str(e)}")

        except Exception as e:
            print(f"Не удалось найти YouTube видео для {song['song_name']}: {str(e)}")

    except Exception as e:
        print(f"Не удалось обработать песню {song['song_name']}: {str(e)}")

    # Сохраняем JSON после каждой итерации
    with open(json_file_path, "w", encoding="utf-8") as file:
        json.dump(songs, file, ensure_ascii=False, indent=4)
        print(f"JSON файл обновлен после обработки {song['song_name']}.")

# Закрываем браузер
driver.quit()


Успешный вход.
Кнопка ревизий для Master of Puppets нажата.
Скачивание файла для Master of Puppets начато: https://gp.songsterr.com/export.136683885.gp
YouTube видео найдено: https://www.youtube.com/embed/E0ozmU9cJDg?controls=0&disablekb=1&showinfo=0&rel=0&modestbranding=0&origin=https%3A%2F%2Fwww.songsterr.com&fs=0&cc_load_policy=0&iv_load_policy=3&enablejsapi=1&start=0&widgetid=1
Ссылка на YouTube для Master of Puppets найдена: https://www.youtube.com/embed/E0ozmU9cJDg?controls=0&disablekb=1&showinfo=0&rel=0&modestbranding=0&origin=https%3A%2F%2Fwww.songsterr.com&fs=0&cc_load_policy=0&iv_load_policy=3&enablejsapi=1&start=0&widgetid=1
JSON файл обновлен после обработки Master of Puppets.
Кнопка ревизий для Nothing Else Matters нажата.
Скачивание файла для Nothing Else Matters начато: https://gp.songsterr.com/export.113739081.gp
YouTube видео найдено: https://www.youtube.com/embed/tAGnKpE4NCI?controls=0&disablekb=1&showinfo=0&rel=0&modestbranding=0&origin=https%3A%2F%2Fwww.songsterr.co

Скачаем ютуб аудио

In [ ]:
import json
import os
from yt_dlp import YoutubeDL

# Путь к вашему JSON файлу
json_path = "songs_data.json"

# Папка для сохранения аудио
download_folder = "downloads_audio"

# Проверка, существует ли папка, если нет — создаем
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Открытие JSON файла и обработка данных
with open(json_path, "r", encoding="utf-8") as file:
    songs = json.load(file)

# Настройка опций для загрузки аудио
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': os.path.join(download_folder, '%(title)s.%(ext)s'),
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

# Функция для скачивания аудио с YouTube
def download_audio_from_youtube(url, title, download_folder):
    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
            print(f"Скачан аудиофайл для '{title}' по ссылке {url}")
        return True  # Успешное скачивание
    except Exception as e:
        print(f"Ошибка скачивания для '{title}' по ссылке {url}: {e}")
        return False  # Скачивание не удалось

# Итерация по песням и скачивание аудио
for song in songs:
    song_name = song["song_name"]
    youtube_link = song.get("youtube_link")

    # Проверка наличия ссылки и попытка скачивания
    if not youtube_link or not download_audio_from_youtube(youtube_link, song_name, download_folder):
        song["download_failed"] = True  # Пометка в JSON, если скачивание не удалось

# Сохранение обновленного JSON с пометками
with open(json_path, "w", encoding="utf-8") as file:
    json.dump(songs, file, ensure_ascii=False, indent=4)


Ошибся с названиями песен, приходится переименовывать

In [5]:
import json
import os
from yt_dlp import YoutubeDL

# Путь к вашему JSON файлу
json_path = "songs_data.json"

# Папка, где находятся скачанные аудио файлы
download_folder = "downloads_audio"

# Открытие JSON файла и обработка данных
with open(json_path, "r", encoding="utf-8") as file:
    songs = json.load(file)

# Настройка опций для получения информации о видео
ydl_opts = {
    'quiet': True,  # Отключает вывод информации для упрощения вывода
}

# Функция для получения оригинального названия с YouTube и переименования
def rename_downloaded_file(song, download_folder):
    youtube_link = song.get("youtube_link")
    song_name = song["song_name"]
    
    # Если ссылка отсутствует, пропускаем
    if not youtube_link:
        print(f"Ссылка отсутствует для '{song_name}', пропускаем.")
        return
    
    try:
        # Получение информации о видео
        with YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_link, download=False)
            original_title = info.get('title')
        
        # Поиск файла, соответствующего оригинальному названию видео
        for filename in os.listdir(download_folder):
            if original_title in filename:
                # Определение нового имени файла
                new_filename = f"{song_name}.mp3"
                new_filepath = os.path.join(download_folder, new_filename)
                
                # Переименование файла
                os.rename(os.path.join(download_folder, filename), new_filepath)
                print(f"Файл '{filename}' переименован в '{new_filename}'")
                return True

        print(f"Файл для '{song_name}' не найден в папке.")
        return False

    except Exception as e:
        # Обработка ошибок и переход к следующей записи
        print(f"Ошибка для '{song_name}': {e}")
        return False

# Итерация по песням и переименование файлов
start_parsing = False
for song in songs:
    if song["song_name"] == "Tears In Heaven":
        start_parsing = True

    if start_parsing:
        rename_downloaded_file(song, download_folder)


Файл для 'Tears In Heaven' не найден в папке.


ERROR: [youtube] W2BO3UTOhZ8: Video unavailable. This video is not available


Ошибка для 'Highway Star': ERROR: [youtube] W2BO3UTOhZ8: Video unavailable. This video is not available
Файл для 'Dream On' не найден в папке.
Файл 'Rammstein - Sonne (Audio).mp3' переименован в 'Sonne.mp3'
Файл 'Tears Don't Fall.mp3' переименован в 'Tears Don't Fall.mp3'
Файл 'Nutshell.mp3' переименован в 'Nutshell.mp3'
Файл для 'Another One Bites the Dust' не найден в папке.
Файл 'Ain't Talkin' 'Bout Love (2015 Remaster).mp3' переименован в 'Ain't Talkin' 'Bout Love.mp3'
Файл 'Franz Ferdinand - Take Me Out (Official Video).mp3' переименован в 'Take Me Out.mp3'
Файл 'Boulevard of Broken Dreams.mp3' переименован в 'Boulevard Of Broken Dreams.mp3'
Файл 'TOOL - Pneuma (Audio).mp3' переименован в 'Pneuma.mp3'
Файл для 'Am I Evil' не найден в папке.
Файл 'Three Days Grace - Animal I Have Become.mp3' переименован в 'Animal I Have Become.mp3'
Файл 'Ozzy Osbourne - Bark at the Moon (Lyrics).mp3' переименован в 'Bark At The Moon.mp3'
Файл 'Earth, Wind & Fire - September (HQ).mp3' переименован 

ERROR: [youtube] HvDBAozIWK4: Video unavailable. This video contains content from SME, who has blocked it in your country on copyright grounds


Ошибка для 'Waking The Demon': ERROR: [youtube] HvDBAozIWK4: Video unavailable. This video contains content from SME, who has blocked it in your country on copyright grounds
Файл 'Oasis - Supersonic (Official HD Remastered Video).mp3' переименован в 'Supersonic.mp3'
Файл 'Nirvana - Drain You [Nevermind] [HQ Sound].mp3' переименован в 'Drain You.mp3'
Файл 'Rosemary.mp3' переименован в 'Rosemary.mp3'
Файл 'The Beatles - Come Together.mp3' переименован в 'Come Together.mp3'
Ссылка отсутствует для 'Rock You Like A Hurricane', пропускаем.
Файл 'Megadeth - Sweating Bullets (remastered 2004).mp3' переименован в 'Sweating Bullets.mp3'
Файл 'Weezer - Undone - The Sweater Song.mp3' переименован в 'Undone (The Sweater Song).mp3'
Файл 'Pantera - 5 Minutes Alone (Official Music Video) [4K Remaster].mp3' переименован в '5 Minutes Alone.mp3'
Файл 'Led Zeppelin - Since I've Been Loving You (Official Audio).mp3' переименован в 'Since I've Been Loving You.mp3'
Файл 'Gorillaz - Feel Good Inc HD.mp3' пере

...

Ошибка для 'Heavy Is the Crown': ERROR: [youtube] ZAt8oxY0GQo: Video unavailable. The uploader has not made this video available in your country
Файл 'TOOL - The Pot (Audio).mp3' переименован в 'The Pot.mp3'
Ссылка отсутствует для 'A Tout Le Monde', пропускаем.
Файл 'Сентябрь.mp3' переименован в 'Сентябрь.mp3'
Файл 'Lamb of God - Walk with Me In Hell (Official Video).mp3' переименован в 'Walk With Me In Hell.mp3'
Файл 'Wake Up Dead (2004 Remaster).mp3' переименован в 'Wake Up Dead.mp3'
Файл 'Pierce The Veil - Bulls in the Bronx.mp3' переименован в 'Bulls In The Bronx.mp3'
Файл 'Scorpions - No One Like You (Official Video).mp3' переименован в 'No One Like You.mp3'
Файл 'Whiplash (Remastered).mp3' переименован в 'Whiplash.mp3'
Файл 'Jump In The Fire (Remastered).mp3' переименован в 'Jump In The Fire.mp3'
Файл 'Save Me.mp3' переименован в 'Save Me.mp3'
Файл 'Europa (Earth's Cry Heaven's Smile).mp3' переименован в 'Europa.mp3'
Файл 'Stone Temple Pilots - Interstate Love Song (Official Lyri

ERROR: [youtube] THPpOWXOUnw: Video unavailable. This video contains content from WMG, who has blocked it in your country on copyright grounds


Ошибка для 'Jesus Of Suburbia': ERROR: [youtube] THPpOWXOUnw: Video unavailable. This video contains content from WMG, who has blocked it in your country on copyright grounds
Файл 'Stand By Me.mp3' переименован в 'Stand By Me (Bass Tabs Only).mp3'
Файл 'The Strokes - Reptilia (Subtitulada Esp - Lyrics).mp3' переименован в 'Reptilia.mp3'
Файл 'John Petrucci - Glasgow Kiss.mp3' переименован в 'Glasgow Kiss.mp3'
Файл 'Jet - Are You Gonna Be My Girl.mp3' переименован в 'Are You Gonna Be My Girl.mp3'
Файл 'My Friend Of Misery.mp3' переименован в 'My Friend Of Misery.mp3'
Файл 'Thank You for the Venom.mp3' переименован в 'Thank You For The Venom.mp3'
Файл 'Arctic Monkeys - Fluorescent Adolescent (Official Video).mp3' переименован в 'Fluorescent Adolescent.mp3'
Файл '7. Jackson 5 - I Want You Back.mp3' переименован в 'I Want You Back.mp3'
Файл 'Green Day - Longview - [HQ].mp3' переименован в 'Longview.mp3'
Файл 'Muse - Knights Of Cydonia  (Video).mp3' переименован в 'Knights Of Cydonia.mp3'


ERROR: [youtube] UV6kNHjEu2g: Video unavailable. This video has been removed by the uploader


Ошибка для '星座になれたら(If I could be a constellation)': ERROR: [youtube] UV6kNHjEu2g: Video unavailable. This video has been removed by the uploader
Файл 'The Smashing Pumpkins - Cherub Rock.mp3' переименован в 'Cherub Rock.mp3'
Файл 'Damage, Inc. (Remastered).mp3' переименован в 'Damage, Inc.mp3'
Файл 'Lying From You - Linkin Park (Meteora).mp3' переименован в 'Lying From You.mp3'
Файл 'Rage Against The Machine - Bulls On Parade (Official HD Video).mp3' переименован в 'Bulls On Parade.mp3'
Файл 'Revolution Is My Name (2003 Remaster).mp3' переименован в 'Revolution Is My Name.mp3'
Файл 'Velvet Revolver - Slither (Official Video).mp3' переименован в 'Slither.mp3'
Файл 'M.I.A..mp3' переименован в 'MIA.mp3'
Файл 'For the Love of God.mp3' переименован в 'For The Love of God.mp3'
Файл 'Dread and the Fugitive Mind (2019 - Remaster).mp3' переименован в 'Dread And The Fugitive Mind.mp3'
Файл 'Led Zeppelin - Over The Hills And Far Away.mp3' переименован в 'Over The Hills And Far Away.mp3'
Файл 'Ra